In [32]:
from pathlib import Path
from pydantic import BaseModel
from typing import Any
from neo4j import GraphDatabase
from dotenv import load_dotenv
import json
import os

load_dotenv()

True

In [31]:
base_path = Path().cwd().parent
storage_path = base_path / Path('raw/entity')

neo4j_uri = os.environ['neo4j_uri']
neo4j_username = os.environ['neo4j_username']
neo4j_password = os.environ['neo4j_password']
neo4j_dbname = os.environ['neo4j_dbname']

In [46]:
class Neo4jGraph:

    def __init__(self, neo4j_uri:str, neo4j_username:str, neo4j_password:str, db:str)->None:
        self.uri  = neo4j_uri
        self.auth = (neo4j_username, neo4j_password)
        self.db = db

    def query(self, query:str, params:dict):
        driver = GraphDatabase.driver(self.uri, auth=self.auth)
        with driver.session(database=self.db) as session:
            result = session.run(query, params)
            return result
        
class Node(BaseModel):
    id:int
    label: list[str]
    properties:dict[str, Any]

class Relation(BaseModel):
    id:str
    label:str
    properties:dict[str, Any]


In [28]:
graph = Neo4jGraph(
    neo4j_uri,
    neo4j_username,
    neo4j_password,
    neo4j_dbname,
)

In [57]:
def merge_node(graph:Neo4jGraph, nodes:list[Node]):
    res = graph.query(
        "UNWIND $data as row "
        "CALL apoc.merge.node("
        "row.label, "
        "row.id, "
        "row.properties, "
        "row.properties ) "
        "YIELD node "
        "RETURN node"
        ,
        {
            "data":[
                node.__dict__ for node in nodes
            ]
        }
    )
    return res


In [48]:
with open(storage_path / Path('anime/page-1.json'), 'r') as fp:
    data = json.load(fp)

In [49]:
doc_list = data.get('Page').get('media')

In [64]:
import duckdb

In [50]:
nodes = []
node_labels  = ['Anime']
for doc in doc_list:
    id_ = doc.pop('id')
    node = Node(id=id_, label=node_labels, properties=doc)
    nodes.append(node)


In [60]:
res = merge_node(graph, nodes)

In [63]:
[i for i in res]

ResultFailedError: The result has failed. Either this result or another result in the same transaction has encountered an error.